# Libs

In [134]:
import requests
import pandas as pd
import zipfile
import io
import time
from datetime import datetime, timedelta

# Convert US format date (MM/DD/YYYY) to datetime object

In [129]:
def convert_to_iso(us_date, us_time="07:00"):
    # Convert US format date (MM/DD/YYYY) and time (HH:MM) to datetime object
    date_time_str = f"{us_date} {us_time}"
    date_object = datetime.strptime(date_time_str, "%m/%d/%Y %H:%M")
    
    # Convert datetime object to ISO 8601 format
    iso_date = date_object.strftime("%Y%m%dT%H:%M")

    # Manually append "-0000"
    iso_date += "-0000"
    
    return iso_date

def date_range(start, end):
    start_date = datetime.strptime(start, "%m/%d/%Y")
    end_date = datetime.strptime(end, "%m/%d/%Y")
    
    current_date = start_date
    
    while current_date < end_date:
        next_date = current_date + timedelta(days=30)
        if next_date > end_date:
            next_date = end_date

        yield convert_to_iso(current_date.strftime("%m/%d/%Y")), convert_to_iso(next_date.strftime("%m/%d/%Y"))
        current_date = next_date

# Preset

In [159]:
start_date = "11/09/2022"
end_date = "06/09/2023"
market = "DAM"  # DAM for day ahead market
node = "SLAP_PGNP-APND" # the node ID

# Collecting history for the given tameframe

In [160]:
# Initialize an empty DataFrame to store all data
df_total = pd.DataFrame()

for start, end in date_range(start_date, end_date):
    print(f"Start: {start}, End: {end}")

    # The base URL for the API
    base_url = "http://oasis.caiso.com/oasisapi/SingleZip"

    # The API parameters (these will depend on the specific data you want)
    params = {
        "queryname": "PRC_LMP", # the type of data you want
        "startdatetime": start, # the start time in ISO8601 format like 20230305T07:00-0000
        "enddatetime": end, # the end time in ISO8601 format like 20230404T07:00-0000
        "market_run_id": market,
        "node": node,
        "version": "1", # the version of the data, 1 is the latest
        "resultformat": "6", #retuns CSV file
    }

    # Send the GET request
    response = requests.get(base_url, params=params)

    # Make sure the request was successful
    response.raise_for_status()

    # Read the ZIP file from the response
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # Get the name of the first (and only) file in the archive
    csv_file_name = zip_file.namelist()[0]

    # Open that file
    csv_file = zip_file.open(csv_file_name)

    # Load the CSV data into a DataFrame
    df = pd.read_csv(csv_file)

    # Concatenate the new data to the existing DataFrame
    df_total = pd.concat([df_total, df])
    print('Collected!')

    # Delay for 3 seconds, below 3 seconds I face responce of "too many requests to URL"
    time.sleep(3)

df_total.loc[:, 'INTERVALSTARTTIME_GMT'] = pd.to_datetime(df_total['INTERVALSTARTTIME_GMT'])
df_total = df_total.sort_values('INTERVALSTARTTIME_GMT')
df_total = df_total.reset_index(drop=True)

# df_total now contains all the data

print('Finished!')

Start: 20221109T07:00-0000, End: 20221209T07:00-0000
Collected!
Start: 20221209T07:00-0000, End: 20230108T07:00-0000
Collected!
Start: 20230108T07:00-0000, End: 20230207T07:00-0000
Collected!
Start: 20230207T07:00-0000, End: 20230309T07:00-0000
Collected!
Start: 20230309T07:00-0000, End: 20230408T07:00-0000
Collected!
Start: 20230408T07:00-0000, End: 20230508T07:00-0000
Collected!
Start: 20230508T07:00-0000, End: 20230607T07:00-0000
Collected!
Start: 20230607T07:00-0000, End: 20230609T07:00-0000
Collected!
Finished!


# Saving df_total dataframe into CSV file

In [ ]:
df_total.to_csv('df_total.csv', index=False)

# Cleaning of the dataframe and saveing updated dataframe into df_total_cleaned.csv

In [154]:
df_total_cleaned = df_total[df_total['LMP_TYPE'] == 'LMP'] # removeing all rows where not LMP in the column LMP_TYPE
df_total_cleaned = df_total_cleaned.reset_index(drop=True) # reseting index
df_total_cleaned.to_csv('df_total_cleaned.csv', index=False) # saving csv file
df_total_cleaned


,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,OPR_INTERVAL,NODE_ID_XML,NODE_ID,NODE,MARKET_RUN_ID,LMP_TYPE,XML_DATA_ITEM,PNODE_RESMRID,GRP_TYPE,POS,MW,GROUP
0,2020-06-09 07:00:00+00:00,2020-06-09T08:00:00-00:00,2020-06-09,1,0,SLAP_PGNP-APND,SLAP_PGNP-APND,SLAP_PGNP-APND,DAM,LMP,LMP_PRC,SLAP_PGNP-APND,ALL_APNODES,0,24.12264,1
1,2020-06-09 08:00:00+00:00,2020-06-09T09:00:00-00:00,2020-06-09,2,0,SLAP_PGNP-APND,SLAP_PGNP-APND,SLAP_PGNP-APND,DAM,LMP,LMP_PRC,SLAP_PGNP-APND,ALL_APNODES,0,23.85739,1
2,2020-06-09 09:00:00+00:00,2020-06-09T10:00:00-00:00,2020-06-09,3,0,SLAP_PGNP-APND,SLAP_PGNP-APND,SLAP_PGNP-APND,DAM,LMP,LMP_PRC,SLAP_PGNP-APND,ALL_APNODES,0,24.37201,1
3,2020-06-09 10:00:00+00:00,2020-06-09T11:00:00-00:00,2020-06-09,4,0,SLAP_PGNP-APND,SLAP_PGNP-APND,SLAP_PGNP-APND,DAM,LMP,LMP_PRC,SLAP_PGNP-APND,ALL_APNODES,0,24.35549,1
4,2020-06-09 11:00:00+00:00,2020-06-09T12:00:00-00:00,2020-06-09,5,0,SLAP_PGNP-APND,SLAP_PGNP-APND,SLAP_PGNP-APND,DAM,LMP,LMP_PRC,SLAP_PGNP-APND,ALL_APNODES,0,24.46661,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26275,2023-06-09 02:00:00+00:00,2023-06-09T03:00:00-00:00,2023-06-08,20,0,SLAP_PGNP-APND,SLAP_PGNP-APND,SLAP_PGNP-APND,DAM,LMP,LMP_PRC,SLAP_PGNP-APND,ALL_APNODES,0,58.99722,71
26276,2023-06-09 03:00:00+00:00,2023-06-09T04:00:00-00:00,2023-06-08,21,0,SLAP_PGNP-APND,SLAP_PGNP-APND,SLAP_PGNP-APND,DAM,LMP,LMP_PRC,SLAP_PGNP-APND,ALL_APNODES,0,59.93073,71
26277,2023-06-09 04:00:00+00:00,2023-06-09T05:00:00-00:00,2023-06-08,22,0,SLAP_PGNP-APND,SLAP_PGNP-APND,SLAP_PGNP-APND,DAM,LMP,LMP_PRC,SLAP_PGNP-APND,ALL_APNODES,0,53.40177,71
26278,2023-06-09 05:00:00+00:00,2023-06-09T06:00:00-00:00,2023-06-08,23,0,SLAP_PGNP-APND,SLAP_PGNP-APND,SLAP_PGNP-APND,DAM,LMP,LMP_PRC,SLAP_PGNP-APND,ALL_APNODES,0,45.16895,71
